# 算法题 python实现 深入理解

In [1]:
import time
import tracemalloc
from typing import Callable, Any, Optional
import functools

In [2]:
def performance_analyzer(
    repeat: int = 1,          # 函数执行次数（取平均，减小误差）
    print_details: bool = True # 是否打印详细结果
) -> Callable:
    """
    函数性能分析装饰器：计算运行时间（ms）和内存占用（MB）
    
    :param repeat: 执行次数，取平均时间（建议耗时短的函数设更大值，如1000）
    :param print_details: 是否打印分析结果
    :return: 装饰后的函数
    """
    def decorator(func: Callable) -> Callable:
        @functools.wraps(func)  # 保留原函数元信息
        def wrapper(*args, **kwargs) -> Any:
            # 1. 初始化性能指标
            total_time = 0.0       # 总运行时间（秒）
            peak_memory = 0.0      # 内存峰值（MB）
            func_result: Optional[Any] = None

            # 2. 多次执行函数（减小计时误差）
            for _ in range(repeat):
                # 计时开始
                start_time = time.perf_counter()
                
                # 内存追踪开始
                tracemalloc.start()
                
                # 执行原函数
                func_result = func(*args, **kwargs)
                
                # 内存追踪结束
                current_mem, peak_mem = tracemalloc.get_traced_memory()
                tracemalloc.stop()
                
                # 计时结束
                end_time = time.perf_counter()
                
                # 累加时间 & 更新峰值内存
                total_time += (end_time - start_time)
                current_peak_mb = peak_mem / (1024 * 1024)  # 转换为MB
                if current_peak_mb > peak_memory:
                    peak_memory = current_peak_mb

            # 3. 计算平均指标
            avg_time_ms = (total_time / repeat) * 1000  # 转换为毫秒
            peak_memory_mb = round(peak_memory, 6)      # 保留6位小数
            avg_time_ms = round(avg_time_ms, 6)

            # 4. 打印结果（可选）
            if print_details:
                print("=" * 60)
                print(f"函数 {func.__name__} 性能分析结果")
                print("=" * 60)
                print(f"执行次数: {repeat} 次")
                print(f"平均运行时间: {avg_time_ms} 毫秒 (ms)")
                print(f"内存峰值占用: {peak_memory_mb} 兆字节 (MB)")
                print("=" * 60)

            # 5. 返回函数执行结果 + 性能指标（可选）
            # 如需在代码中获取指标，可返回元组：(结果, 时间ms, 内存MB)
            return func_result, avg_time_ms, peak_memory_mb

        return wrapper
    return decorator

## 704.二分查找

给定一个 n 个元素有序的（升序）整型数组 nums 和一个目标值 target  ，写一个函数搜索 nums 中的 target，如果目标值存在返回下标，否则返回 -1

Case1：
```
输入: nums = [-1,0,3,5,9,12], target = 9     
输出: 4       
解释: 9 出现在 nums 中并且下标为 4  
```

Case2:
```
输入: nums = [-1,0,3,5,9,12], target = 2     
输出: -1        
解释: 2 不存在 nums 中因此返回 -1 
```


提示：

- 你可以假设 nums 中的所有元素是不重复的。
- n 将在 [1, 10000]之间。
- nums 的每个元素都将在 [-9999, 9999]之间。

**解**
1. 用enumerate似乎太偷懒了
2. 就用正常的二分法
3. 边界处需要注意开闭区间

In [16]:
@performance_analyzer(repeat=1)
def search(nums: list[int], target: int) -> int:
    """leetcode解法 二分法 默认左右闭区间"""
    left = 0
    right = len(nums) - 1
    while left <= right:
        middle = left + (right - left) // 2

        if nums[middle] > target: # 左区间
            right = middle - 1
        elif nums[middle] < target: # 右区间
            left = middle + 1
        else:
            return middle
    return -1

In [15]:
result = search(
    nums = [-1,0,3,5,9,12],
    target=9
)
result

函数 search 性能分析结果
执行次数: 1 次
平均运行时间: 0.012699 毫秒 (ms)
内存峰值占用: 0.000183 兆字节 (MB)


(4, 0.012699, 0.000183)

In [28]:
"""
更偷懒的方法
"""
nums = [-1,0,3,5,9,12]
target=9
nums.index(target)

4

## 27. 移除元素

给你一个数组 nums 和一个值 val，你需要 原地 移除所有数值等于 val 的元素，并返回移除后数组的新长度。

不要使用额外的数组空间，你必须仅使用 O(1) 额外空间并原地修改输入数组。

元素的顺序可以改变。你不需要考虑数组中超出新长度后面的元素。

示例 1: 给定 nums = [3,2,2,3], val = 3, 函数应该返回新的长度 2, 并且 nums 中的前两个元素均为 2。 你不需要考虑数组中超出新长度后面的元素。

示例 2: 给定 nums = [0,1,2,2,3,0,4,2], val = 2, 函数应该返回新的长度 5, 并且 nums 中的前五个元素为 0, 1, 3, 0, 4。

你不需要考虑数组中超出新长度后面的元素。

**解答**
1. 很偷懒的方法, 直接list.remove(target), 但可能不符合时间复杂度的要求
2. 快慢指针，快指针用来比较、慢指针用来记录新的数组

In [4]:
nums = [0,1,2,3,3,0,4,2]
target = 2

In [5]:
nums.remove(target)
nums

[0, 1, 3, 3, 0, 4, 2]

In [10]:
@performance_analyzer(repeat=1)
def remove_elements(nums: list[int], val:int) -> int:
    """移除nums(list)中元素值为val(int)的元素 并返回移除后数组的长度"""
    temp_list = []
    for num in nums:
        if not num == val:
            temp_list.append(num)
    
    return len(temp_list)
        

In [11]:
nums = [0,1,2,3,3,0,4,2]
target = 2

remove_elements(nums=nums, val=target)

函数 remove_elements 性能分析结果
执行次数: 1 次
平均运行时间: 0.054496 毫秒 (ms)
内存峰值占用: 0.00071 兆字节 (MB)


(6, 0.054496, 0.00071)

In [29]:
@performance_analyzer(repeat=1)
def removeElement(nums: list[int], val: int) -> int:
    """leetcode解法 快慢指针 快指针用来比较、慢指针用来记录新的数组"""
    fast = 0
    slow = 0
    size = len(nums)
    while fast < size:
        if nums[fast] != val:
            # print(f"nums[fast] is {nums[fast]}")
            nums[slow] = nums[fast] # 感觉没有什么意义 返回输出是长度而不是数组
            # print(f"nums[slow] is {nums[slow]}")
            slow += 1
        fast += 1
    return slow

In [28]:
nums = [0,1,2,3,3,0,4,2]
target = 2

removeElement(nums, target)

函数 removeElement 性能分析结果
执行次数: 1 次
平均运行时间: 0.008 毫秒 (ms)
内存峰值占用: 0.0 兆字节 (MB)


(6, 0.008, 0.0)

## 977. 有序数组的平方

给你一个按 非递减顺序 排序的整数数组 nums，返回 每个数字的平方 组成的新数组，要求也按 非递减顺序 排序。

```
输入：nums = [-4,-1,0,3,10]
输出：[0,1,9,16,100]
解释：平方后，数组变为 [16,1,0,9,100]，排序后，数组变为 [0,1,9,16,100]
示例 2：

输入：nums = [-7,-3,2,3,11]
输出：[4,9,9,49,121]
```

**解答**
1. 直觉是暴力法，平方完，然后排序
2. 解析一下就是，数组是有序的（包含负数），平方后最大值不是在最左边就是在最右边，不可能在中间
3. 定义双指针, 从两边从中间数

In [30]:
@performance_analyzer(repeat=1)
def sort_squares(nums: list[int]) -> list[int]:
    """数组平方后排序"""
    tmp_list = [x ** 2 for x in nums]
    tmp_list = sorted(tmp_list)

    return tmp_list

In [31]:
nums =  [-4,-1,0,3,10]
sort_squares(nums=nums)

函数 sort_squares 性能分析结果
执行次数: 1 次
平均运行时间: 0.031998 毫秒 (ms)
内存峰值占用: 0.00042 兆字节 (MB)


([0, 1, 9, 16, 100], 0.031998, 0.00042)

In [33]:
@performance_analyzer(repeat=1)
def sortedSquares(nums: list[int]) -> list[int]:
    """leetcode解法 双指针法 从中间开始数"""
    left = 0
    right = len(nums) - 1
    i = len(nums) - 1
    res = [float('inf')] * len(nums)
    while left <= right:
        if nums[left] ** 2 < nums[right] ** 2:
            res[i] = nums[right] ** 2
            right -= 1
        else:
            res[i] = nums[left] ** 2
            left += 1
        i -= 1
    return res

In [34]:
nums =  [-4,-1,0,3,10]
sortedSquares(nums)

函数 sortedSquares 性能分析结果
执行次数: 1 次
平均运行时间: 0.016499 毫秒 (ms)
内存峰值占用: 4.6e-05 兆字节 (MB)


([0, 1, 9, 16, 100], 0.016499, 4.6e-05)

## 4. 寻找两个正序数组的中位数  (还没做出来)

给定两个大小分别为 m 和 n 的正序（从小到大）数组 nums1 和 nums2。请你找出并返回这两个正序数组的 中位数 。

算法的时间复杂度应该为 O(log (m+n)) 。

 

示例 1：

输入：nums1 = [1,3], nums2 = [2]
输出：2.00000
解释：合并数组 = [1,2,3] ，中位数 2
示例 2：

输入：nums1 = [1,2], nums2 = [3,4]
输出：2.50000
解释：合并数组 = [1,2,3,4] ，中位数 (2 + 3) / 2 = 2.5

In [49]:
@performance_analyzer(repeat=1)
def find_median(nums1:list[int], nums2: list[int]):
    """暴力解法"""
    tmp_list = nums1 + nums2
    print(tmp_list)
    tmp_list = sorted(tmp_list)
    total_len = len(tmp_list)
    if total_len % 2 == 0:
        return (tmp_list[total_len // 2 - 1] + tmp_list[total_len // 2]) / 2
    else:
        return tmp_list[total_len // 2]
    

In [51]:
@performance_analyzer(repeat=1)
def findSortedMedian(nums1:list[int], nums2: list[int]):
    # 确保nums1是更短的数组
    if len(nums1) > len(nums2):
        nums1, nums2 = nums2, nums1
    m, n = len(nums1), len(nums2)
    
    total_len = m + n
    left, right = 0, m
    left_total = (total_len + 1) // 2
    while left <= right:
        # i: nums1左半部分元素个数 j:nums2左半部分元素个数
        i = (left + right) // 2
        j = left_total -i

        nums1_left = nums1[i-1] if i > 0 else float('-inf')
        nums1_right = nums1[i] if i < m else float('inf')
        nums2_left = nums2[j-1] if j > 0 else float('-inf')
        nums2_right = nums2[j] if j < n else float('inf')

        if nums1_left > nums2_right:
            right = i - 1
        elif nums2_left > nums1_right:
            left = i + 1
        else:
            if total_len & 2 == 1:
                return float(max(nums1_left, nums2_left))
            else:
                return (max(nums1_left + nums2_left) + min(nums1_right + nums2_right)) / 2.0

In [48]:
nums1 = [1,2]
nums2 = [2,3]

find_median(nums1, nums2)


[1, 2, 2, 3]
函数 find_median 性能分析结果
执行次数: 1 次
平均运行时间: 0.136898 毫秒 (ms)
内存峰值占用: 0.002477 兆字节 (MB)


(2.0, 0.136898, 0.002477)

In [52]:
nums1 = [1,2]
nums2 = [2,3]

findSortedMedian(nums1, nums2)


TypeError: 'int' object is not iterable

## 1. 两数之和

给定一个整数数组 nums 和一个整数目标值 target，请你在该数组中找出 和为目标值 target  的那 两个 整数，并返回它们的数组下标。

你可以假设每种输入只会对应一个答案，并且你不能使用两次相同的元素。

你可以按任意顺序返回答案。

 

示例 1：

输入：nums = [2,7,11,15], target = 9
输出：[0,1]
解释：因为 nums[0] + nums[1] == 9 ，返回 [0, 1] 。
示例 2：

输入：nums = [3,2,4], target = 6
输出：[1,2]
示例 3：

输入：nums = [3,3], target = 6
输出：[0,1]

In [3]:
@performance_analyzer(repeat=1)
def two_Sum(nums: list[int], target: int) -> list[int]:
    """两数之和"""
    for i, num in enumerate(nums):
        if target - num in nums[i+1:]:
            return [i, nums[i+1:].index(target - num) + i + 1]

In [4]:
nums = [2,7,11,15]
target = 9

two_Sum(nums, target)

函数 two_Sum 性能分析结果
执行次数: 1 次
平均运行时间: 0.2102 毫秒 (ms)
内存峰值占用: 0.000565 兆字节 (MB)


([0, 1], 0.2102, 0.000565)

In [ ]:
@performance_analyzer(repeat=1)
def two_sum_hash(nums: list[int], target: int) -> list[int]:
    """
    两数之和 hash解法
    hash_map中 num是key index是value
    """
    hash_map = {}
    for i, num in enumerate(nums):
        if (target - num) in hash_map:
            return [hash_map[target - num], i]
        hash_map[num] = i
        print(hash_map)
    return []
    


In [10]:
nums = [2,7,11,15]
target = 9

two_sum_hash(nums, target)

{2: 0}
函数 two_sum_hash 性能分析结果
执行次数: 1 次
平均运行时间: 0.083408 毫秒 (ms)
内存峰值占用: 0.002183 兆字节 (MB)


([0, 1], 0.083408, 0.002183)

## 11. 盛最多水的容器

给定一个长度为 n 的整数数组 height 。有 n 条垂线，第 i 条线的两个端点是 (i, 0) 和 (i, height[i]) 。

找出其中的两条线，使得它们与 x 轴共同构成的容器可以容纳最多的水。

返回容器可以储存的最大水量。

输入：[1,8,6,2,5,4,8,3,7]
输出：49 
解释：图中垂直线代表输入数组 [1,8,6,2,5,4,8,3,7]。在此情况下，容器能够容纳水（表示为蓝色部分）的最大值为 49。
示例 2：

输入：height = [1,1]
输出：1



In [5]:
@performance_analyzer(repeat=1)
def max_area(nums:list[int]) -> int:
    """
    盛最多水的容器 -> 面积最大
    长width: j - i    宽height: min(height[i], height[j])
    双指针 向中间靠拢 
    """
    total_length = len(nums)
    if total_length < 2:
        return 0
    
    max_area = 0
    left_ptr, right_ptr = 0, total_length - 1
    
    while left_ptr < right_ptr:
        width = right_ptr - left_ptr
        height = min(nums[left_ptr], nums[right_ptr])
        area = width * height

        max_area = max(max_area, area)

        if nums[left_ptr] < nums[right_ptr]:
            left_ptr += 1
        else:
            right_ptr -= 1
    return max_area            


In [6]:
heights = [1,8,6,2,5,4,8,3,7]
max_area(heights)


函数 max_area 性能分析结果
执行次数: 1 次
平均运行时间: 0.3551 毫秒 (ms)
内存峰值占用: 4.6e-05 兆字节 (MB)


(49, 0.3551, 4.6e-05)

## 14. 最长公共前缀

编写一个函数来查找字符串数组中的最长公共前缀。

如果不存在公共前缀，返回空字符串 ""。

 

示例 1：

输入：strs = ["flower","flow","flight"]
输出："fl"
示例 2：

输入：strs = ["dog","racecar","car"]
输出：""
解释：输入不存在公共前缀。

In [9]:
@performance_analyzer(repeat=1)
def longest_common_prefix(strs: list[str]) -> str:
    if not strs:
        return ""
    
    """
    将第一个字符串str的全部元素作为前缀
    将后面的字符串与第一个字符串比较 元素不断缩减
    """
    prefix = strs[0]
    for i in range(1, len(strs)):
        while not strs[i].startswith(prefix):
            prefix = prefix[:-1]

            if not prefix:
                return ""
    return prefix

In [10]:
strs = ["flower","flow","flight"]
longest_common_prefix(strs)

函数 longest_common_prefix 性能分析结果
执行次数: 1 次
平均运行时间: 0.355907 毫秒 (ms)
内存峰值占用: 0.000148 兆字节 (MB)


('fl', 0.355907, 0.000148)